In [3]:
# --- PARALLELIZATION: SALES ANALYSIS EDITION ---


%pip install -U langchain_openai langgraph
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGSMITH_API_KEY")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

# --- Overview ---
# We’ll demonstrate parallel execution where multiple LLM calls
# analyze product sales in parallel (like workers).

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from typing_extensions import TypedDict
from typing import Annotated
import operator
from pydantic import BaseModel
from IPython.display import Image

# --- Model ---
model = ChatOpenAI(model="gpt-4o", temperature=0)

# --- Prompts ---
products_prompt = """Generate a list of 4 products in the category: {category}."""
sales_analysis_prompt = """Analyze sales performance, growth, and challenges for the product: {product}.
Write 3 concise bullet points summarizing the insights."""

# --- Data Models ---
class Products(BaseModel):
    products: list[str]

class ProductSummary(BaseModel):
    summary: str

# --- State Definition ---
class OverallState(TypedDict):
    category: str
    products: list
    summaries: Annotated[list, operator.add]

# --- Node 1: Generate product list ---
def generate_products(state: OverallState):
    prompt = products_prompt.format(category=state["category"])
    response = model.with_structured_output(Products).invoke(prompt)
    return {"products": response.products}

# --- Node 2: Parallel worker to analyze each product ---
class ProductState(TypedDict):
    product: str

def analyze_product(state: ProductState):
    prompt = sales_analysis_prompt.format(product=state["product"])
    response = model.with_structured_output(ProductSummary).invoke(prompt)
    return {"summaries": [f"{state['product']}: {response.summary}"]}

# --- Control logic for parallel sends ---
def continue_to_analysis(state: OverallState):
    return [Send("analyze_product", {"product": p}) for p in state["products"]]

# --- Node 3: Combine summaries (no reduce logic needed here) ---
def summarize_all(state: OverallState):
    print("\n=== Combined Sales Summaries ===\n")
    for s in state["summaries"]:
        print(s)
    print("\nAnalysis complete.")
    return {}

# --- GRAPH CONSTRUCTION ---
graph = StateGraph(OverallState)
graph.add_node("generate_products", generate_products)
graph.add_node("analyze_product", analyze_product)
graph.add_node("summarize_all", summarize_all)
graph.add_edge(START, "generate_products")
graph.add_conditional_edges("generate_products", continue_to_analysis, ["analyze_product"])
graph.add_edge("analyze_product", "summarize_all")
graph.add_edge("summarize_all", END)

# --- Compile and Visualize ---
app = graph.compile()
Image(app.get_graph().draw_mermaid_png())

# --- RUN ---
for s in app.stream({"category": "Electronics"}):
    print(s)


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


{'generate_products': {'products': ["1. **Sony WH-1000XM4 Wireless Noise-Canceling Headphones**  \n   - Features: Industry-leading noise cancellation, up to 30 hours of battery life, touch sensor controls, and voice assistant compatibility.\n   - Price: $349.99\n   - Description: Experience the ultimate in sound quality and comfort with these top-of-the-line headphones, perfect for music lovers and frequent travelers.\n\n2. **Apple iPad Pro 12.9-inch (2023)**  \n   - Features: M2 chip, Liquid Retina XDR display, 5G capability, and Apple Pencil support.\n   - Price: Starting at $1,099\n   - Description: The latest iPad Pro offers unparalleled performance and versatility, making it ideal for creative professionals and tech enthusiasts.\n\n3. **Samsung Galaxy S23 Ultra Smartphone**  \n   - Features: 200MP camera, 6.8-inch Dynamic AMOLED 2X display, Snapdragon 8 Gen 2 processor, and 5000mAh battery.\n   - Price: Starting at $1,199\n   - Description: Capture stunning photos and enjoy a seam